In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [2]:
conf = (SparkConf().setAppName('CreateTableInSQL').setMaster('yarn') 
    .set("spark.sql.adaptive.enabled", False)  )
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("app_id".ljust(40), spark.sparkContext.applicationId)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/27 20:01:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/27 20:01:54 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
25/02/27 20:01:54 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


app_id                                   application_1727681258360_0133


In [4]:

sql = """
   select * from values 
    (1, 'Anna', True), 
    (2, 'Bob', True), 
    (3, 'Cindy', False) as (emp_id, emp_name, promoted)
"""
df = spark.sql(sql)
df.show()

root
 |-- emp_id: integer (nullable = false)
 |-- emp_name: string (nullable = false)
 |-- promoted: boolean (nullable = false)

+------+--------+--------+
|emp_id|emp_name|promoted|
+------+--------+--------+
|     1|    Anna|    true|
|     2|     Bob|    true|
|     3|   Cindy|   false|
+------+--------+--------+



In [3]:
df1 = spark.read.option('inferSchema', True).load(\
      '/user/ssenigov/my-spark-storage/tblcustomers', \
    format='csv')
print(df1.count())
df2 = spark.read.option('inferSchema', True).load(\
      '/user/ssenigov/my-spark-storage/tblcustomers_bucketed', \
    format='csv')
print(df2.count())

1000000


1000000


In [3]:
df12 = spark.read.option('inferSchema', True).load(\
      ['/user/ssenigov/my-spark-storage/tblcustomers',
       '/user/ssenigov/my-spark-storage/tblcustomers_bucketed'], \
    format='csv')
print(df12.count())

2000000


In [38]:


import datetime
from pyspark.sql import SQLContext

def input_event_paths(base_path, date, depth):
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    return [f"{base_path}/date={(dt-datetime.timedelta(days=x)).strftime('%Y-%m-%d')}" for x in range(int(depth))]

ll = input_event_paths('base', '2025-01-01', 5)
ll

ll = ['/user/ssenigov/my-spark-storage/tblcustomers',
       '/user/ssenigov/my-spark-storage/tblcustomers_bucketed']
print(*ll)

sc = spark.sparkContext
sql = SQLContext(sc)

df_ = spark.read.option('inferSchema', True).text(*ll)
print(df12.count())

/user/ssenigov/my-spark-storage/tblcustomers /user/ssenigov/my-spark-storage/tblcustomers_bucketed


2000000


In [10]:
file_name = "/user/ssenigov/my-spark-storage/tblcustomers_bucketed/" + \
            "part-00000-54dab98f-c3d5-43c7-85bd-38344a8f7eb9_00000.c000.csv"

df3 = spark.read.option('inferSchema', True).load(file_name, format='csv')
print(df3.count())

41657


In [12]:
df4 = spark.table('tblcustomers_bucketed')
dff.count()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `tblcustomers_bucketed` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [tblcustomers_bucketed], [], false


In [62]:
spark.stop()